In [ ]:
elastic_rods_dir = '../../elastic_rods/python/'
weaving_dir = '../'
import os
import os.path as osp
import sys; sys.path.append(elastic_rods_dir); sys.path.append(weaving_dir)
import numpy as np, elastic_rods, linkage_vis
import numpy.linalg as la
from bending_validation import suppress_stdout as so
import matplotlib.pyplot as plt
from elastic_rods import EnergyType, InterleavingType

In [ ]:
# Reloadable libraries
import importlib

# weaving
import analysis_helper, ribbon_linkage_helper, mesh_vis, linkage_utils, compute_curve_from_curvature, importlib
importlib.reload(analysis_helper)
importlib.reload(ribbon_linkage_helper)
importlib.reload(mesh_vis)
importlib.reload(linkage_utils)
importlib.reload(compute_curve_from_curvature)
from analysis_helper import (compare_turning_angle,
                            is_on_sphere, 
                            get_distance_to_center_scalar_field, 
                            plot_curvatures, 
                            get_curvature_scalar_field,
                            construct_elastic_rod_loop_from_rod_segments, 
                            concatenate_rod_properties_from_rod_segments, 
                            compute_min_distance_rigid_transformation)
from ribbon_linkage_helper import (update_rest_curvature, 
                                   set_ribbon_linkage,
                                   export_linkage_geometry_to_obj,
                                   write_linkage_ribbon_output_florin)

from compute_curve_from_curvature import (match_geo_curvature_and_edge_len, get_all_curve_pattern)
from linkage_utils import order_segments_by_ribbons, get_turning_angle_and_length_from_ordered_rods

import vis.fields
import matplotlib.cm as cm

In [ ]:
import time

In [ ]:
# Set to True in the options below if you already optimized a specific linkage
# and you would like to reuse the optimized linkage. Loading is performed in
# place of the full design optimization.
# NOTE: Doesn't seem to work at the moment, leave at False
LOAD_OPTIMIZED_DOFS = False

# Set to False if strips cannot be labeled into families for a model
USE_FAMILY_LABEL = True

In [ ]:
# Sphere Parameters
default_camera_parameters = ((3.466009282140468, -4.674139805388271, -2.556131049738206), (-0.21402574298422497, -0.06407538766530313, -0.9747681088523519),(0.1111, 0.1865, 0.5316))
RIBBON_CS = [1, 10]
ISO_CS = [4, 4]
MODEL_PATH = osp.join(weaving_dir + 'models/equal_edge_atI.obj')
MODEL_NAME = "equal_edge_atI"
SUBDIVISION_RESOLUTION = 20
REGULARIZATION_WEIGHT = 0.05
SMOOTHING_WEIGHT = 0
INPUT_SURFACE_PATH = osp.join(weaving_dir + 'surface_models/centered_sphere_100mm.obj')
RIBBON_NAME = "sphere_strip"

In [ ]:
# # Ellipsoidal Parameters
# default_camera_parameters = ((3.466009282140468, -4.674139805388271, -2.556131049738206), (-0.21402574298422497, -0.06407538766530313, -0.9747681088523519),(0.1111, 0.1865, 0.5316))
# RIBBON_CS = [1, 10]
# ISO_CS = [4, 4]
# MODEL_PATH = osp.join(weaving_dir + 'models/three_axis_ellipsoidal_equal_edge_atI.obj')
# INPUT_SURFACE_PATH = osp.join(weaving_dir + 'surface_models/three_axis_ellipsoid.obj')
# MODEL_NAME = 'three_axis_ellipsoidal_from_equal_edge_atI'
# SUBDIVISION_RESOLUTION = 20
# REGULARIZATION_WEIGHT = 0.1
# RIBBON_NAME = "ellipsoidal_strip"

In [ ]:
# # Manta Ray Parameters
# default_camera_parameters = ((3.466009282140468, -4.674139805388271, -2.556131049738206), (-0.21402574298422497, -0.06407538766530313, -0.9747681088523519),(0.1111, 0.1865, 0.5316))
# RIBBON_CS = [12, 8]
# ISO_CS = [4, 4]
# MODEL_PATH = osp.join(weaving_dir + 'models/manta_ray.obj')
# MODEL_NAME = "manta_ray"
# SUBDIVISION_RESOLUTION = 20
# REGULARIZATION_WEIGHT = 0.05
# INPUT_SURFACE_PATH = osp.join(weaving_dir + 'surface_models/centered_sphere_100mm.obj')

In [ ]:
# # Single Linkage Parameters
# default_camera_parameters = ((3.466009282140468, -4.674139805388271, -2.556131049738206), (-0.21402574298422497, -0.06407538766530313, -0.9747681088523519),(0.1111, 0.1865, 0.5316))
# RIBBON_CS = [0.001, 0.01]
# ISO_CS = [0.01, 0.01]
# MODEL_PATH = osp.join('../../examples/' + 'single_linkage.obj')
# SUBDIVISION_RESOLUTION = 5
# MODEL_NAME = "single_linkage"
# REGULARIZATION_WEIGHT = 0.05
# INPUT_SURFACE_PATH = osp.join(weaving_dir + 'surface_models/single_linkage_target.obj')

In [ ]:
# # Pseudosphere
# default_camera_parameters = ((3.466009282140468, -4.674139805388271, -2.556131049738206), (-0.21402574298422497, -0.06407538766530313, -0.9747681088523519),(0.1111, 0.1865, 0.5316))
# RIBBON_CS = [0.001, 0.01]
# ISO_CS = [1, 1]
# MODEL_NAME = "pseudo_sphere"
# MODEL_PATH = osp.join(weaving_dir + 'models/{}.obj'.format(MODEL_NAME))
# INPUT_SURFACE_PATH =  osp.join(weaving_dir + 'surface_models/pseudo_sphere/pseudo_sphere_surface.obj')
# SUBDIVISION_RESOLUTION = 20
# REGULARIZATION_WEIGHT = 0.0001

In [ ]:
# # Sphere to hemoglobin sequence
# default_camera_parameters = ((3.466009282140468, -4.674139805388271, -2.556131049738206), (-0.21402574298422497, -0.06407538766530313, -0.9747681088523519),(0.1111, 0.1865, 0.5316))
# RIBBON_CS = [0.01, 0.1]
# ISO_CS = [0.04, 0.04]
# MODEL_PATH = osp.join(weaving_dir + 'models/sphere_to_hemoglobin/5.obj')
# SUBDIVISION_RESOLUTION = 20
# REGULARIZATION_WEIGHT = 0.05
# MODEL_NAME = './results/sphere_to_hemoglobin_5'

In [ ]:
# Regular Torus Parameters
default_camera_parameters = ((3.466009282140468, -4.674139805388271, -2.556131049738206), (-0.21402574298422497, -0.06407538766530313, -0.9747681088523519),(0.1111, 0.1865, 0.5316))
default_camera_parameters_2 = ((1.6535518732236205, -2.1434469534967877, -5.518813874877859),
 (-0.5096009366331774, 0.449950326402865, -0.733441973950591),
 (0.1111, 0.1865, 0.5316))
RIBBON_CS = [4, 40]
ISO_CS = [1, 1]
MODEL_PATH = osp.join(weaving_dir + 'models/regular_torus.obj')
INPUT_SURFACE_PATH = osp.join(weaving_dir + 'surface_models/regular_torus.obj')
MODEL_NAME = 'regular_torus'
SUBDIVISION_RESOLUTION = 20
REGULARIZATION_WEIGHT = 0.5
RIBBON_NAME = 'regular_torus_strip'

In [ ]:
# # GMO Torus Parameters
# default_camera_parameters = ((3.466009282140468, -4.674139805388271, -2.556131049738206), (-0.21402574298422497, -0.06407538766530313, -0.9747681088523519),(0.1111, 0.1865, 0.5316))
# RIBBON_CS = [1, 40]
# MODEL_PATH = osp.join(weaving_dir + 'models/gmo_torus/gmo_torus.obj')
# INPUT_SURFACE_PATH = osp.join(weaving_dir + 'surface_models/gmo_torus.obj')
# MODEL_NAME = 'gmo_torus'
# SUBDIVISION_RESOLUTION = 20
# REGULARIZATION_WEIGHT = 1
# RIBBON_NAME = 'gmo_torus_strip'

In [ ]:
# # Pseudosphere 400
# default_camera_parameters = ((3.466009282140468, -4.674139805388271, -2.556131049738206), (-0.21402574298422497, -0.06407538766530313, -0.9747681088523519),(0.1111, 0.1865, 0.5316))
# RIBBON_CS = [0.1, 4]
# MODEL_NAME = "pseudo_sphere_400"
# MODEL_PATH = osp.join(weaving_dir + 'models/{}.obj'.format(MODEL_NAME))
# SUBDIVISION_RESOLUTION = 20
# SMOOTHING_WEIGHT = 100
# REGULARIZATION_WEIGHT = 1
# RIBBON_NAME = "pseudo_sphere_400_strip"

In [ ]:
# # Sphere to simple hemoglobin sequence
# default_camera_parameters = ((1.455670753946072, -1.6783546089674806, -6.2366218363694985),
#  (0.9765104371975557, -0.011543058475481151, -0.21535938299796684),
#  (0.1111, 0.1865, 0.5316))
# RIBBON_CS = [0.01, 0.4]
# MODEL_PATH = osp.join(weaving_dir + 'models/simple_sphere_to_hemoglobin/5.obj')
# INPUT_SURFACE_PATH = osp.join(weaving_dir + 'surface_models/sphere_to_hemoglobin/5.obj')
# SUBDIVISION_RESOLUTION = 20
# REGULARIZATION_WEIGHT = 0.001
# SMOOTHING_WEIGHT = 0
# MODEL_NAME = './results/simple_sphere_to_hemoglobin'
# RIBBON_NAME = 'simple_hemoglobin_strip'

In [ ]:
# # Pseudosphere Small
# default_camera_parameters = ((3.466009282140468, -4.674139805388271, -2.556131049738206), (-0.21402574298422497, -0.06407538766530313, -0.9747681088523519),(0.1111, 0.1865, 0.5316))
# RIBBON_CS = [0.005, 0.03]
# MODEL_NAME = "small_pseudo_sphere"
# MODEL_PATH = osp.join(weaving_dir + 'models/{}.obj'.format(MODEL_NAME))
# SUBDIVISION_RESOLUTION = 20
# SMOOTHING_WEIGHT = 10
# REGULARIZATION_WEIGHT = 0
# INPUT_SURFACE_PATH = osp.join(weaving_dir + 'surface_models/pseudo_sphere/{}.obj'.format(MODEL_NAME))
# RIBBON_NAME = "small_pseudo_sphere_strip"

In [ ]:
# # Pseudosphere Biaxial
# default_camera_parameters = ((3.466009282140468, -4.674139805388271, -2.556131049738206), (-0.21402574298422497, -0.06407538766530313, -0.9747681088523519),(0.1111, 0.1865, 0.5316))
# RIBBON_CS = [0.005, 0.03]
# MODEL_NAME = "biaxial_pseudosphere"
# MODEL_PATH = osp.join(weaving_dir + 'models/{}.obj'.format(MODEL_NAME))
# SUBDIVISION_RESOLUTION = 20
# SMOOTHING_WEIGHT = 10
# REGULARIZATION_WEIGHT = 0
# INPUT_SURFACE_PATH = osp.join(weaving_dir + 'surface_models/pseudo_sphere/{}.obj'.format('small_pseudo_sphere'))
# RIBBON_NAME = "biaxial_pseudosphere_strip"

In [ ]:
# # Freeform 5 
# default_camera_parameters = ((3.466009282140468, -4.674139805388271, -2.556131049738206), (-0.21402574298422497, -0.06407538766530313, -0.9747681088523519),(0.1111, 0.1865, 0.5316))
# RIBBON_CS = [15, 150]
# MODEL_NAME = "freeform_5"
# MODEL_PATH = osp.join(weaving_dir + 'models/{}.obj'.format(MODEL_NAME))
# SUBDIVISION_RESOLUTION = 5
# SMOOTHING_WEIGHT = 10
# REGULARIZATION_WEIGHT = 0
# INPUT_SURFACE_PATH = osp.join(weaving_dir + 'surface_models/{}.obj'.format(MODEL_NAME))

# RIBBON_NAME = "freeform_5_strip"

In [ ]:
# # Klein bottle
# default_camera_parameters = ((3.466009282140468, -4.674139805388271, -2.556131049738206), (-0.21402574298422497, -0.06407538766530313, -0.9747681088523519),(0.1111, 0.1865, 0.5316))
# RIBBON_CS = [0.001, 0.01]
# MODEL_NAME = "kleinbottle"
# MODEL_PATH = osp.join(weaving_dir + 'models/{}.obj'.format(MODEL_NAME))
# SUBDIVISION_RESOLUTION = 20
# SMOOTHING_WEIGHT = 10
# REGULARIZATION_WEIGHT = 0
# INPUT_SURFACE_PATH = osp.join(weaving_dir + 'surface_models/{}.obj'.format(MODEL_NAME))
# LOAD_OPTIMIZED_DOFS = False
# USE_FAMILY_LABEL = False
# RIBBON_NAME = "kleinbottle_strip"

In [ ]:
# # Bird
# default_camera_parameters = ((3.466009282140468, -4.674139805388271, -2.556131049738206), (-0.21402574298422497, -0.06407538766530313, -0.9747681088523519),(0.1111, 0.1865, 0.5316))
# RIBBON_CS = [100, 1000]
# MODEL_NAME = "bird"
# MODEL_PATH = osp.join(weaving_dir + 'models/{}.obj'.format(MODEL_NAME))
# SUBDIVISION_RESOLUTION = 20
# SMOOTHING_WEIGHT = 10
# REGULARIZATION_WEIGHT = 0
# INPUT_SURFACE_PATH = osp.join(weaving_dir + 'surface_models/{}.obj'.format(MODEL_NAME))
# RIBBON_NAME = "bird_strip"

In [ ]:
# # Close Bird
# default_camera_parameters = ((3.466009282140468, -4.674139805388271, -2.556131049738206), (-0.21402574298422497, -0.06407538766530313, -0.9747681088523519),(0.1111, 0.1865, 0.5316))
# RIBBON_CS = [100, 1000]
# MODEL_NAME = "bird_close_beak"
# MODEL_PATH = osp.join(weaving_dir + 'models/{}.obj'.format(MODEL_NAME))
# SUBDIVISION_RESOLUTION = 20
# SMOOTHING_WEIGHT = 10
# REGULARIZATION_WEIGHT = 0
# INPUT_SURFACE_PATH = osp.join(weaving_dir + 'surface_models/{}.obj'.format('bird'))
# RIBBON_NAME = "bird_close_beak_strip"

In [ ]:
# # Open Bird
# default_camera_parameters = ((3.466009282140468, -4.674139805388271, -2.556131049738206), (-0.21402574298422497, -0.06407538766530313, -0.9747681088523519),(0.1111, 0.1865, 0.5316))
# RIBBON_CS = [100, 1000]
# MODEL_NAME = "bird_open_beak"
# MODEL_PATH = osp.join(weaving_dir + 'models/{}.obj'.format(MODEL_NAME))
# SUBDIVISION_RESOLUTION = 20
# SMOOTHING_WEIGHT = 10
# REGULARIZATION_WEIGHT = 0
# INPUT_SURFACE_PATH = osp.join(weaving_dir + 'surface_models/{}.obj'.format('bird'))
# RIBBON_NAME = "bird_open_beak_strip"

In [ ]:
def initialize_linkage(surface_path = INPUT_SURFACE_PATH, useCenterline = True, cross_section = RIBBON_CS, subdivision_res = SUBDIVISION_RESOLUTION, model_path = MODEL_PATH):
    l = elastic_rods.SurfaceAttractedLinkage(surface_path, useCenterline, model_path, subdivision_res, False, InterleavingType.weaving)
    l.setMaterial(elastic_rods.RodMaterial('rectangle', 2000, 0.3, cross_section, stiffAxis=elastic_rods.StiffAxis.D1))
    l.set_holdClosestPointsFixed(True);
    l.set_attraction_tgt_joint_weight(0.01);
    l.attraction_weight = 100;
    return l

In [ ]:
def initialize_normal_linkage(cross_section = RIBBON_CS, subdivision_res = SUBDIVISION_RESOLUTION, model_path = MODEL_PATH):
    l = elastic_rods.RodLinkage(model_path, subdivision_res, False)
    l.setMaterial(elastic_rods.RodMaterial('rectangle', 2000, 0.3, cross_section, stiffAxis=elastic_rods.StiffAxis.D1))
    return l

In [ ]:
def design_parameter_solve(l,regularization_weight = 0.1, smoothing_weight = 1):
    design_opts = elastic_rods.NewtonOptimizerOptions()
    design_opts.niter = 10000
    design_opts.verbose = 1
    l.set_design_parameter_config(use_restLen = True, use_restKappa = True)
    elastic_rods.designParameter_solve(l, design_opts, regularization_weight = 0.0, smoothing_weight = 0.001)
    l.set_design_parameter_config(use_restLen = True, use_restKappa = True)

In [ ]:
def get_linkage_eqm(l, opt, cam_param = default_camera_parameters, target_surf = None):
    elastic_rods.compute_equilibrium(l, options = opt)
    if (target_surf is None):
        view = linkage_vis.LinkageViewer(l, width=1024, height=640)
    else:
        view = linkage_vis.LinkageViewerWithSurface(l, target_surf, width=1024, height=640)
    view.setCameraParams(cam_param)
    return l, view

In [ ]:
OPTS = elastic_rods.NewtonOptimizerOptions()
OPTS.gradTol = 1e-6
OPTS.verbose = 1;
OPTS.beta = 1e-8
OPTS.niter = 100
OPTS.verboseNonPosDef = False

### Generate straight linkage equilibrium for testing later

In [ ]:
straight_linkage = initialize_linkage(surface_path = INPUT_SURFACE_PATH, useCenterline = True, model_path = MODEL_PATH, cross_section = RIBBON_CS, subdivision_res = SUBDIVISION_RESOLUTION)

In [ ]:
view = linkage_vis.LinkageViewer(straight_linkage, width=1024, height=640)
view.setCameraParams(default_camera_parameters)
view.show()

In [ ]:
# temp_view = linkage_vis.LinkageViewer(straight_linkage, width=1024, height=640)
# temp_view.setCameraParams(default_camera_parameters)

In [ ]:
# temp_view.show()

In [ ]:
# export_linkage_geometry_to_obj(straight_linkage, '{}_intialized.obj'.format(MODEL_NAME))

In [ ]:
straight_linkage.attraction_weight = 0
straight_linkage, initial_view = get_linkage_eqm(straight_linkage, cam_param = default_camera_parameters, opt = OPTS)

In [ ]:
initial_view.show()

In [ ]:
straight_rod_dof = straight_linkage.getDoFs()

In [ ]:
elastic_rods.compute_equilibrium(straight_linkage, options = OPTS)

In [ ]:
initial_view.update()

### Create Surface Attracted Linkage and solve for design parameters with fixed joint positions

In [ ]:
with so(): linkage = initialize_linkage(surface_path = INPUT_SURFACE_PATH, useCenterline = True, model_path = MODEL_PATH, cross_section = RIBBON_CS, subdivision_res = SUBDIVISION_RESOLUTION)
# with so(): linkage = initialize_normal_linkage(model_path = MODEL_PATH, cross_section = RIBBON_CS, subdivision_res = SUBDIVISION_RESOLUTION)
save_tgt_joint_pos = linkage.jointPositions();

In [ ]:
with so(): design_parameter_solve(linkage, regularization_weight = 0.0, smoothing_weight = 0.001)

In [ ]:
start_time = time.time()
with so(): linkage, view = get_linkage_eqm(linkage, cam_param = default_camera_parameters, opt = OPTS, target_surf=INPUT_SURFACE_PATH)
print('compute equilibrium takes: ', time.time() - start_time)
view.show()

In [ ]:
# Optionally configure the visualization colors/transparency
view.viewOptions[view.ViewType.LINKAGE].color = 'lightgreen'
view.viewOptions[view.ViewType.LINKAGE].transparent = False
view.viewOptions[view.ViewType.SURFACE].transparent = True
view.viewOptions[view.ViewType.SURFACE].color = 'gray'
view.applyViewOptions()

In [ ]:
la.norm(linkage.gradient())

In [ ]:
linkage.numJoints()

In [ ]:
OPTS.niter = 200
linkage.set_design_parameter_config(use_restLen = True, use_restKappa = True)
useCenterline = True
optimizer = elastic_rods.WeavingOptimization(linkage, INPUT_SURFACE_PATH, useCenterline, equilibrium_options=OPTS, pinJoint = 0, useFixedJoint = False)
optimizer.set_target_joint_position(save_tgt_joint_pos)
view.update()

In [ ]:
optimizer.rl_regularization_weight = 0
optimizer.smoothing_weight = 10
optimizer.beta = 500000.0
optimizer.gamma = 1
algorithm = elastic_rods.WeavingOptAlgorithm.NEWTON_CG
def update_viewer():
    view.update()

In [ ]:
if not LOAD_OPTIMIZED_DOFS:
    optimizer.WeavingOptimize(algorithm, 2000, 1.0, 1e-2, update_viewer)
else:
    loadedDoFs = np.load("dof_files/{}.npy".format(MODEL_NAME))
    linkage.setExtendedDoFs(loadedDoFs)
    view.update()

In [ ]:
linkage, view = get_linkage_eqm(linkage, cam_param = default_camera_parameters, opt = OPTS)
view.update()

In [ ]:
## If you want to solve again after loading optimized DOFs (e.g. if the code changed)
# algorithm = elastic_rods.WeavingOptAlgorithm.NEWTON_CG
# optimizer.WeavingOptimize(algorithm, 2000, 1.0, 1e-2, update_viewer)

In [ ]:
# Store DoFs to file
if not os.path.exists("dof_files"):
    os.makedirs("dof_files")
rawDoFs = linkage.getExtendedDoFs()
np.save("dof_files/{}.npy".format(MODEL_NAME), rawDoFs)

In [ ]:
optimizer.setLinkageAttractionWeight(1e-16)

In [ ]:
optimizer.set_holdClosestPointsFixed(False)

### Apply perturbation 

In [ ]:
# linesearch_weaver = optimizer.get_linesearch_weaver()
# linesearch_weaver.setDoFs(straight_rod_dof)

In [ ]:
# perturbation_test_view = linkage_vis.LinkageViewer(linesearch_weaver, width=1024, height=640)
# perturbation_test_view.setCameraParams(default_camera_parameters)
# perturbation_test_view.show()

In [ ]:
# linesearch_weaver.attraction_weight

In [ ]:
# with so(): elastic_rods.compute_equilibrium(linesearch_weaver, options = OPTS)

In [ ]:
# perturbation_test_view.update()

In [ ]:
# export_linkage_geometry_to_obj(linkage, '{}_optimized.obj'.format(MODEL_NAME))

In [ ]:
# view.getCameraParams()

In [ ]:
# view.setCameraParams(default_camera_parameters)

In [ ]:
# import vis.fields
# import matplotlib.cm as cm
# sf = vis.fields.ScalarField(linkage, 0.5 * np.ones((linkage.numSegments(), linkage.segment(0).rod.numVertices())), colormap=cm.viridis, vmin = 0, vmax = 1)
# view.update(scalarField=sf)
# default_camera_parameters = ((3.466009282140468, -4.674139805388271, -2.556131049738206), (-0.21402574298422497, -0.06407538766530313, -0.9747681088523519),(0.1111, 0.1865, 0.5316))
# view.setCameraParams(default_camera_parameters)

### Get laser cutting pattern

In [ ]:
# if not os.path.exists(RIBBON_NAME):
#     os.makedirs(RIBBON_NAME)
# get_all_curve_pattern(linkage, RIBBON_CS[1], SUBDIVISION_RESOLUTION, RIBBON_NAME, image_type='svg')

### Generate Output needed by Florin for Rendering

In [ ]:
ribbons = order_segments_by_ribbons(linkage)
write_linkage_ribbon_output_florin(linkage, ribbons, SUBDIVISION_RESOLUTION, RIBBON_NAME, USE_FAMILY_LABEL)